In [1]:
import os
import os.path as osp
import sys
from itertools import combinations
sys.path.append(osp.abspath('..'))

import numpy as np
from tqdm import tqdm_notebook
from sklearn.preprocessing import normalize
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC as SVCMClass
from sklearn.metrics import accuracy_score

import config
from datasets.gtzan import GTZAN_SPEC

%load_ext autoreload
%autoreload 2

In [2]:
# Random seeds
np.random.seed(1234)
MIN_SEGMENTS = 2

In [8]:
train_set = GTZAN_SPEC(phase='val', test_size=0.1, val_size=0.4,
                       min_segments=MIN_SEGMENTS, randomized=True)
test_set = GTZAN_SPEC(phase='test', test_size=0.1, val_size=0.4, min_segments=MIN_SEGMENTS)

print('Train:', len(train_set))
print('Test:', len(test_set))
print('Shape:', train_set[0][0].shape)

Train: 400
Test: 100
Shape: (2, 221, 301)


In [4]:
def concatenate_data(*datasets):
    X = []
    for dataset in datasets:
        for sample_idx, (x, _) in tqdm_notebook(enumerate(dataset), total=len(dataset)):
            for x_ in x:
                X.append(x_)
    X = np.concatenate(X, axis=1)
    return X

Ewhiten = None
if not osp.exists('pca_whiten_mat.npy'):
    X = concatenate_data(train_set)

    # PCA whiten
    Xcov = X.dot(X.T) / X.shape[1]
    num_features = Xcov.shape[0]
    eigen_values, eigen_vectors = np.linalg.eigh(Xcov)

    E = eigen_vectors[:, num_features:num_features-NUM_PRINCIPAL_FEATURES-1:-1]
    S = np.diag(eigen_values[num_features:num_features-NUM_PRINCIPAL_FEATURES-1:-1])

    Xpc = E.T.dot(X)
    Xrec = E.dot(Xpc)

    Ewhiten = np.diag(1. / np.sqrt((np.diag(S)+SIGMA_PC))).dot(E.T)
    Eunwhiten = E.dot(np.diag(np.sqrt(np.diag(S)+SIGMA_PC)))

    Xrec2 = Eunwhiten.dot(Ewhiten).dot(X)
    
    plt.figure(figsize=(16, 32))
    plt.subplot(1, 3, 1)
    plt.imshow(X[:, :301], cmap='hot')
    plt.subplot(1, 3, 2)
    plt.imshow(Xrec[:, :301], cmap='hot')
    plt.subplot(1, 3, 3)
    plt.imshow(Xrec2[:, :301], cmap='hot')
    plt.show()
    
    with open('pca_whiten_mat.npy', 'wb') as f:
        np.save(f, Ewhiten)
else:
    with open('pca_whiten_mat.npy', 'rb') as f:
        Ewhiten = np.load(f)
assert Ewhiten is not None

In [9]:
# aggregate over all frames, calculate simple summary statistics
# such as average, max or standard deviation for each channel
x_train = np.empty((400 * MIN_SEGMENTS, 80 * 301))
y_train = np.empty((400 * MIN_SEGMENTS,))
x_test = np.empty((100 * MIN_SEGMENTS, 80 * 301))
y_test = np.empty((100 * MIN_SEGMENTS,))
 

def populate_features(src_dataset, dst_x, dst_y):
    for sample_idx, (x, y) in tqdm_notebook(enumerate(src_dataset), total=len(src_dataset)):
        for segment_idx, x_ in enumerate(x):
            idx = sample_idx * MIN_SEGMENTS + segment_idx
            dst_x[idx] = Ewhiten.dot(x_).flatten()
            dst_y[idx] = y

populate_features(train_set, x_train, y_train)
populate_features(test_set, x_test, y_test)

In [10]:
params = {
    'C': [10**x for x in range(-3, 3)],
    #'gamma': [10**x for x in range(-4, 0)],
    'random_state': [1234],
}

rs = GridSearchCV(SVCMClass(), params, n_jobs=-1, verbose=1)
rs.fit(x_train, y_train)
print(rs.best_score_)
print(rs.best_params_)

Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:   32.2s finished


0.37375
{'C': 0.001, 'random_state': 1234}


In [11]:
svm = SVCMClass(**rs.best_params_)
svm.fit(x_train, y_train)

print('Train score:', accuracy_score(y_train, svm.predict(x_train)))
print('Test score:', accuracy_score(y_test, svm.predict(x_test)))

Train score: 0.9975
Test score: 0.385
